<a href="https://colab.research.google.com/github/misrori/scraping_2024/blob/main/session_4_yahoo_finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scrapethat

# https://cryptocointracker.com/yahoo-finance/yahoo-finance-api

### Getting Historical Prices from Yahoo Finance API #v8-finance-chart


In [ ]:
from scrapethat import *
import requests
import json

In [ ]:
data_query = '{"filter":[{"left":"type","operation":"in_range","right":["stock","dr","fund"]},{"left":"subtype","operation":"in_range","right":["common","foreign-issuer","","etf","etf,odd","etf,otc","etf,cfd"]},{"left":"exchange","operation":"in_range","right":["AMEX","NASDAQ","NYSE"]},{"left":"is_primary","operation":"equal","right":true},{"left":"active_symbol","operation":"equal","right":true}],"options":{"lang":"en"},"markets":["america"],"symbols":{"query":{"types":[]},"tickers":[]},"columns":["logoid","name","close","change","change_abs","Recommend.All","volume","Value.Traded","market_cap_basic","price_earnings_ttm","earnings_per_share_basic_ttm","number_of_employees","sector","industry","RSI","SMA50","SMA100","SMA200","BB.lower","BB.upper","Perf.W","Perf.Y","Perf.YTD","Perf.3M","Perf.6M","Perf.1M","High.1M","High.3M","High.6M","price_52_week_high","High.All","description","type","subtype","update_mode","pricescale","minmov","fractional","minmove2","RSI[1]","currency","fundamental_currency_code"],"sort":{"sortBy":"market_cap_basic","sortOrder":"desc"},"range":[0,9000]}'
response = requests.post('https://scanner.tradingview.com/america/scan', data=data_query)
data = response.json()
list_elements = list(map(lambda x:x['d'], data['data'] ))
stock = pd.DataFrame(list_elements)
stock.columns = json.loads(data_query)['columns']


In [ ]:
# select the consumer durabels sector and the top 3 from every industry in cosumers durabels sector

stock_consumer_durables =(
    stock[stock['sector']=="Consumer Durables"]
    .groupby('industry', as_index=False)
    .apply(lambda x: x.nlargest(3, 'market_cap_basic'))
    .reset_index(drop=True)

)


<ipython-input-14-9731f2547621>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(3, 'market_cap_basic'))


In [ ]:
stock_consumer_durables["industry"].unique()

array(['Automotive Aftermarket', 'Electronics/Appliances',
       'Home Furnishings', 'Homebuilding', 'Motor Vehicles',
       'Other Consumer Specialties', 'Recreational Products',
       'Tools & Hardware'], dtype=object)

In [ ]:
stock_consumer_durables.head()

,logoid,name,close,change,change_abs,Recommend.All,volume,Value.Traded,market_cap_basic,price_earnings_ttm,...,type,subtype,update_mode,pricescale,minmov,fractional,minmove2,RSI[1],currency,fundamental_currency_code
0,lkq,LKQ,39.76,3.272727,1.26,0.290909,4066825,1.616970e+08,1.033601e+10,14.723745,...,stock,common,delayed_streaming_900,100,1,false,0,52.294567,USD,USD
1,dorman-products,DORM,141.70,1.613482,2.25,0.603030,222793,3.156977e+07,4.324327e+09,23.736934,...,stock,common,delayed_streaming_900,100,1,false,0,69.963216,USD,USD
2,goodyear-tire-and-rubber,GT,10.13,4.325438,0.42,0.424242,6080801,6.159851e+07,2.886236e+09,NaN,...,stock,common,delayed_streaming_900,100,1,false,0,61.743530,USD,USD
3,sharkninja,SN,102.71,4.443767,4.37,0.536364,1447396,1.486620e+08,1.437285e+10,40.278431,...,stock,common,delayed_streaming_900,100,1,false,0,45.794130,USD,USD
4,roku,ROKU,68.22,-1.416185,-0.98,-0.357576,3993711,2.724510e+08,9.905491e+09,NaN,...,stock,common,delayed_streaming_900,100,1,false,0,43.869158,USD,USD


In [ ]:
ticker = 'TSLA'
range='18y'
interval='1d'

scraper = cloudscraper.create_scraper()
response = scraper.get(f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}?interval={interval}&range={range}")
t= response.json()

df = pd.DataFrame(t['chart']['result'][0]['indicators']['quote'][0])
df['date'] = pd.to_datetime(t['chart']['result'][0]['timestamp'], unit='s').date
df = df[['date', 'open', 'low', 'high', 'close', 'volume']]

df

,date,open,low,high,close,volume
0,2010-06-29,1.266667,1.169333,1.666667,1.592667,281494500
1,2010-06-30,1.719333,1.553333,2.028000,1.588667,257806500
2,2010-07-01,1.666667,1.351333,1.728000,1.464000,123282000
3,2010-07-02,1.533333,1.247333,1.540000,1.280000,77097000
4,2010-07-06,1.333333,1.055333,1.333333,1.074000,103003500
...,...,...,...,...,...,...
3623,2024-11-19,335.760010,332.750000,347.380005,346.000000,88852500
3624,2024-11-20,345.000000,334.299988,346.600006,342.029999,66340700
3625,2024-11-21,343.809998,335.279999,347.989990,339.640015,58011700
3626,2024-11-22,341.089996,337.700012,361.529999,352.559998,89140700


In [ ]:
def get_olhc(ticker, range='30d', interval='1d' ):
    try:

        scraper = cloudscraper.create_scraper()
        response = scraper.get(f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}?interval={interval}&range={range}")
        t= response.json()
        df = pd.DataFrame(t['chart']['result'][0]['indicators']['quote'][0])
        df['date'] = pd.to_datetime(t['chart']['result'][0]['timestamp'], unit='s').date
        df = df[['date', 'open', 'low', 'high', 'close', 'volume']]
        first_close = df['close'].iloc[0]
        df['change'] = (( df['close']  / first_close ) -1) * 100
        df['name']= ticker
        return(df)

    except:
        print('error')
        pass





In [ ]:
get_olhc('TSLA')

,date,open,low,high,close,volume,change,name
0,2024-10-15,220.009995,217.119995,224.259995,219.570007,62988800,0.000000,TSLA
1,2024-10-16,221.399994,218.929993,222.820007,221.330002,49632800,0.801564,TSLA
2,2024-10-17,221.589996,217.899994,222.080002,220.889999,50791800,0.601171,TSLA
3,2024-10-18,220.710007,219.229996,222.279999,220.699997,49611900,0.514638,TSLA
4,2024-10-21,218.899994,215.729996,220.479996,218.850006,47329000,-0.327914,TSLA
5,2024-10-22,217.309998,215.259995,218.220001,217.970001,43268700,-0.728700,TSLA
6,2024-10-23,217.130005,212.110001,218.720001,213.649994,80938900,-2.696185,TSLA
7,2024-10-24,244.679993,242.649994,262.119995,260.480011,204491900,18.631872,TSLA
8,2024-10-25,256.010010,255.320007,269.489990,269.190002,161611900,22.598713,TSLA
9,2024-10-28,270.000000,262.239990,273.540009,262.510010,107653600,19.556406,TSLA


In [ ]:
from tqdm import tqdm

list_of_dfs = list(map(get_olhc, tqdm(stock_consumer_durables["name"])))




100%|██████████| 24/24 [00:03<00:00,  7.55it/s]


In [ ]:
import pandas as pd

all_data = pd.concat(list_of_dfs).reset_index(drop=True)


In [ ]:
# merge sector and industry column
all_data = all_data.merge(stock_consumer_durables[['name', 'sector', 'industry']], on='name')

all_data

,date,open,low,high,close,volume,change,name,sector,industry
0,2024-10-15,39.230000,38.980000,39.910000,39.000000,2168000,0.000000,LKQ,Consumer Durables,Automotive Aftermarket
1,2024-10-16,39.150002,38.950001,39.520000,39.400002,2312600,1.025645,LKQ,Consumer Durables,Automotive Aftermarket
2,2024-10-17,39.450001,39.189999,39.619999,39.389999,3610700,0.999998,LKQ,Consumer Durables,Automotive Aftermarket
3,2024-10-18,39.470001,39.180000,39.669998,39.310001,2220200,0.794875,LKQ,Consumer Durables,Automotive Aftermarket
4,2024-10-21,39.310001,38.799999,39.389999,38.880001,2055300,-0.307690,LKQ,Consumer Durables,Automotive Aftermarket
...,...,...,...,...,...,...,...,...,...,...
715,2024-11-19,26.750000,26.750000,27.780001,27.350000,14900,-14.290190,EML,Consumer Durables,Tools & Hardware
716,2024-11-20,27.000000,26.920000,28.440001,28.209999,12000,-11.595114,EML,Consumer Durables,Tools & Hardware
717,2024-11-21,28.820000,28.110001,28.830000,28.190001,13200,-11.657785,EML,Consumer Durables,Tools & Hardware
718,2024-11-22,28.000000,28.000000,28.940001,28.540001,11000,-10.560949,EML,Consumer Durables,Tools & Hardware


In [ ]:
# creat a plotly to visualize the change the color should be the industry



In [ ]:
import plotly.express as px

fig = px.line(
    all_data,
    x='date',
    y='change',
    color='industry',
    template='plotly_white',
    hover_data=['name', 'change']

)

# Update layout for better visuals
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Percentage Change (%)',
    legend_title='Industry',
    title='Percentage change over time by industry'

)

fig.show()
# ad hover data to ticker
